In [20]:
!docker-compose up -d

clickhouse is up-to-date


In [21]:
import time
import statistics
import uuid
from random import randint

from clickhouse_driver import Client

client = Client(host="localhost") 

In [22]:
client.execute("CREATE DATABASE IF NOT EXISTS compare")

[]

In [23]:
client.execute("DROP TABLE IF EXISTS compare.views")

[]

In [24]:
create_sql = """
CREATE TABLE IF NOT EXISTS compare.views (
    id String,
    user_id String,
    movie_id String,
    timestamp_movie Int64,
    time Int64
) Engine=MergeTree() ORDER BY id
"""
client.execute(create_sql)

[]

In [25]:
def generate_random_data(n_rows: int, iteration: int):
    data = []
    for _ in range(iteration*n_rows):
        timestamp_movie = randint(1, 100)
        time = randint(1000000000, 10000000000)
        data.append((str(uuid.uuid4()), str(uuid.uuid4()), str(uuid.uuid4()), timestamp_movie, time))

    yield data

In [26]:
class profiler():
    def __init__(self):
        self.start = time.time()
        self.all_durations = []
        
    def setup_start_time(self):
        self.start = time.time()

    def checkpoint(self):
        end_time = time.time()
        self.all_durations.append(end_time - self.start)
        self.start = end_time

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        print(f"Median - {statistics.median(self.all_durations)}")
        avg = sum(self.all_durations) / len(self.all_durations)
        print(f"Average - {avg}")
        print(f"Summary - {sum(self.all_durations)}")

In [27]:
# Тестирование вставки по 500 записей
with profiler() as p:
    for data in generate_random_data(500, 2000):
        p.setup_start_time()
        client.execute(
            "INSERT INTO compare.views (id, user_id, movie_id, timestamp_movie, time) VALUES",
            data,
        )
        p.checkpoint()

Median - 1.7571446895599365
Average - 1.7571446895599365
Summary - 1.7571446895599365


In [28]:
# Тестирование одной вставки
with profiler() as p:
    for data in generate_random_data(1, 2000):
        p.setup_start_time()
        client.execute(
            "INSERT INTO compare.views (id, user_id, movie_id, timestamp_movie, time) VALUES",
            data,
        )
        p.checkpoint()

Median - 0.008560419082641602
Average - 0.008560419082641602
Summary - 0.008560419082641602


In [29]:
uuids = [str(u[0]) for u in client.execute("SELECT user_id FROM compare.views LIMIT 100")]

In [30]:
# Тестирование получения данных
with profiler() as p:
    for u in uuids:
        p.setup_start_time()
        client.execute(f"SELECT * FROM compare.views where user_id='{u}'")
        p.checkpoint()

Median - 0.008590340614318848
Average - 0.010925867557525636
Summary - 1.0925867557525635


In [31]:
!docker-compose down

Stopping clickhouse ... 

Stopping clickhouse ... done

Removing clickhouse ... 

Removing clickhouse ... done

Removing network clickhouse_default
